# Developing "Momentum" Trading Strategy
* Many differnt takes on this strategy
* Sometimes referred to as trend following
* Whatever you do and call it, you first need to research potential trading signals

In [7]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from pyalgotrade import strategy


In [15]:
result = pd.read_csv(r'E:\Data_Stock\excel_gvr.csv')
result["Date"] = pd.to_datetime(result["<DTYYYYMMDD>"], format='%Y%m%d')
result['Open'] = pd.to_numeric(result['<OpenFixed>'].astype(str)) * 1000
result['Close'] = pd.to_numeric(result['<CloseFixed>'].astype(str)) * 1000
result['High'] = pd.to_numeric(result['<HighFixed>'].astype(str)) * 1000
result['Low'] = pd.to_numeric(result['<LowFixed>'].astype(str)) * 1000
result['Volume'] = pd.to_numeric(result['<Volume>'].astype(str))
gld = result[['Date','Open','High','Low' ,'Close','Volume']].copy()
gld.to_csv(r'E:\Data_Stock\excel_gvr_2.csv')

In [16]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()


def run_strategy(smaPeriod):
    # Load the bar feed from the CSV file
    feed = quandlfeed.Feed()    
    feed.addBarsFromCSV("orcl", "E:\Data_Stock\excel_gvr_2.csv")
    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "orcl", smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())

run_strategy(15)

Exception: The barfeed doesn't support adjusted close values